# First-Class Functions

A programming language is said to have first-class function if it treats functions as first-class citizens. 

A first-class citizen (sometimes called first-class objects) is an entity that supports all the operations generally available to other entities. For exaple, being passed as an argument, returned from a function, and assigned to a variable. 

In [1]:
def square(x):
    return x * x

f = square(5)
print(square)
print(f)

<function square at 0x00000172C7310D08>
25


You can make **f** take the function by removing the **( )**.

Note: Some people only remove the arguments instead of the parenthesis. 

In [2]:
f = square
print(square)
print(f)

<function square at 0x00000172C7310D08>
<function square at 0x00000172C7310D08>


This means that **square** is a first-class function, and we can use **f** as a square

In [3]:
print(f(5))
print(square(5))

25
25


If a function accepts functions as arguments, or returns a function, this is called **Higher Order Functions**

## Function with function as parameters

In [4]:
def my_map(func, arg_list):
    """
    Takes a function and a list as arguments
    and returns a new list
    """
    result = []
    for i in arg_list:
        result.append(func(i))
    return result
 
# Test it with square func
squares = my_map(square, [1, 2, 3, 4, 5, 6])
print(squares)

[1, 4, 9, 16, 25, 36]


In [5]:
def cube(x):
    return x * x * x

cubes = my_map(cube, [1, 2, 3, 4, 5, 6])
print(cubes)

[1, 8, 27, 64, 125, 216]


## Return a function from another function
Note: To call the return value, which is a **callable** object, use parenthesis. 

In [6]:
def logger(msg):
    def log_message():
        print("Log:", msg)
    # NO parenthesis
    return log_message
# test it
log_hi = logger("Hi!")
# Now run it
log_hi()

Log: Hi!


In [8]:
# another example
def html_tag(tag):
    
    def wrap_text(msg):
        print("<{0}>{1}</{0}>".format(tag, msg))
        
    return wrap_text
# Test it
print_h1 = html_tag("h1")
print_h1("Test Headline")

<h1>Test Headline</h1>


In [10]:
print_p= html_tag("p")
print_p("test paragraph")

<p>test paragraph</p>


### Local functions
That is a function defined inside other functions. Remember the **def** key word is used to define functions. It simply binds the body of the function to a name. Just like any other object. The **def** is execute at runtime. So functions are defined at run time. 

In [11]:
def func1():
    """Regular function"""
    x = 1
    y = 2
    return x + y

def func2():
    """Regular function"""
    def local_func():
        """Local function"""
        a = "hello"
        b = " world"
        return a + b
    x = 1
    y = 2
    return x + y
        

Local function are bound by the **Local Enclosed Global Built-in (LEGB)** rule. 

**Local Function** are NOT members of the containing function in any way. It is just a local name binding. 

* Useful for specilized, one-off functions
* Aid in code organization and readability
* Similar to Lambdas, but more general
    * may contain multiple expression
    * may contain statements

## Closures and Nested Scopes

Remember, a local function can be returned from a function to be used later. 

How the fucntion can relate to the variable that no longer exist? How can the local function operates without that enclosing scope?

The answer is: **Closures**. A closure essentially remembers the object from the enclosing scope that the local function needs. It then keeps them "alive" so that when the local function is executed they can still be used. 

This prevents the auto garbage collection from cleaning the memory. 

Python implements closures with the **dunder closure** method. 

Where is this useful?

## Function Factories

A common use of **closures** is in **so-called function factories**. These functions return another function where the other function are specilized in some way based on the arguments to the "factory". 


Note: see raise_to.py

## Nonlocal keyword
Remember you can use the **global** keyword to bing names to global variables. 

**nonlocal** introduces names from the enclosing namespace into the **local** namespace

# Function Decorators

At a high-level **decorators** are a way to modify or enhance existing functions in a nonintrusive and amintanable way. 

In Python a decorator is an object that is **implemented as callable** that **takes a callable** and **returns a callable** object. 

Think about it as a function that takes a function as argument and returns another function. 

> @my_decorator<br>
> def my_funcion():<br>
>    ...

* Replace, enhance, or modify existing functions
* Does not change the original function definition
* Calling code des not need to change
* Decorator mechanism uses the modified function's original name. 

see: escape_unicode.py

## Classes as decorators
Before we were passing a function as an argument to a function, but how are going to pass a function as argument to a class?

We want to decorate the same function, but now through a **Class Decorator**

In [13]:
def decorator_function(original_function):
    def wrapper_function(*args, **kwargs):
        print('Wrapper executed this before {}'.format(
            original_function.__name__ ))
        return original_function(*args, **kwargs)

    return wrapper_function


class Decorator_class(object):
    def __init__(self, original_function):
        self._original_function = original_function
    # Make it callable    
    def __call__(self, *args, **kwargs):
        print("Call method executed this before {}".format(
        self._original_function.__name__))
        return self._original_function(*args, **kwargs)

#@decorator_function  # function decorator
@Decorator_class   # class decorator
def display():
    print("Display function ran")


#@decorator_function # function decorator
@Decorator_class   # class decorator
def display_info(name, age):
    print("Display_info ran with parameters ({}{})".format(
        name, age ))
    
# test it
display()
display_info("Mario", 12)

Call method executed this before display
Display function ran
Call method executed this before display_info
Display_info ran with parameters (Mario12)


Some people like to decorate with classes. I will be using functions from now on. 

In [17]:
def my_logger(orig_func):
    import logging
    logging.basicConfig(filename='{}.log'.format(
        orig_func.__name__), level = logging.INFO)
    def wrapper(*args, **kwargs):
        logging.info(
        "Ran with args: {}".format(args, kwargs))
        return orig_func(*args, **kwargs) 
    return wrapper

# Decorating
@my_logger
def display_info(name, age):
    print("Display_info ran with parameters ({} {})".format(
        name, age ))
# Test it
display_info("Mary", 33)

Display_info ran with parameters (Mary 33)


Another example is to time how long function run

In [21]:
import time

def my_timer(orig_func):
    def wrapper(*args, **kwargs):
        t1 = time.time()
        result = orig_func(*args, **kwargs)
        t2 = time.time() - t1
        print("{} ran in: {}".format(orig_func.__name__, t2))
        return result
    return wrapper
# Decorating
@my_timer
def display_info(name, age):
    print("Display_info ran with parameters ({} {})".format(
        name, age ))
    time.sleep(1) # sleep
    
# Test it
display_info("Mary", 33)

Display_info ran with parameters (Mary 33)
display_info ran in: 1.0107953548431396


## Multiple decorators
One more example to wrap everything. Apply both logger and timer decorators

In [22]:
# Decorating
@my_timer
@my_logger
def display_info(name, age):
    print("Display_info ran with parameters ({} {})".format(
        name, age ))
    time.sleep(1) # sleep
    
# Test it
display_info("Mary", 33)

Display_info ran with parameters (Mary 33)
wrapper ran in: 1.0022706985473633


It is always a good idea to preserve your original decorator information by using the **functools** module. All we need to do is to decorate all of the wrapper with the **wraps** function

In [23]:
# Decorating
@my_logger
@my_timer
def display_info(name, age):
    print("Display_info ran with parameters ({} {})".format(
        name, age ))
    time.sleep(1) # sleep
    
# Test it
display_info("Mary", 33)

Display_info ran with parameters (Mary 33)
display_info ran in: 1.0059611797332764


In [26]:
import time
from functools import wraps

def my_logger(orig_func):
    import logging
    logging.basicConfig(filename='{}.log'.format(
        orig_func.__name__), level = logging.INFO)
    @wraps(orig_func)
    def wrapper(*args, **kwargs):
        logging.info(
        "Ran with args: {}".format(args, kwargs))
        return orig_func(*args, **kwargs) 
    return wrapper

def my_timer(orig_func):
    @wraps(orig_func)
    def wrapper(*args, **kwargs):
        t1 = time.time()
        result = orig_func(*args, **kwargs)
        t2 = time.time() - t1
        print("{} ran in: {}".format(orig_func.__name__, t2))
        return result
    return wrapper

# Decorating
@my_logger
@my_timer
def display_info(name, age):
    print("Display_info ran with parameters ({} {})".format(
        name, age ))
    time.sleep(1) # sleep
    
# Test it
display_info("Tom", 33)

Display_info ran with parameters (Tom 33)
display_info ran in: 1.013390064239502


# Properties and Class Methods

## Class Attributes
### Class Variables

In [27]:
class Employee:
    def __init__(self, first, last, pay):
        self._first = first
        self._last = last
        self._pay = pay
        self._email = first + '.' + last + '@weber.edu'
        
    def fullname(self):
        return "{} {}".format(self._first, self._last)

In [28]:
# Test it
emp1 = Employee("Juan", "Perez", 50000.00)
emp2 = Employee("Ana", "White", 550000.00)
# Use parenthesis to execute the method
print(emp1.fullname())
print(emp2.fullname())

Juan Perez
Ana White


In [29]:
# Alternative way of calling a method
Employee.fullname(emp1)

'Juan Perez'

Class variables are visible for every instance of the class. Instead of local variables to one instance. Looking at the class above, what information we want to share?



In [31]:
class Employee:
    def __init__(self, first, last, pay):
        self._first = first
        self._last = last
        self._pay = pay
        self._email = first + '.' + last + '@weber.edu'
        
    def fullname(self):
        return "{} {}".format(self._first, self._last)
    
    def apply_raise(self):
        self._pay = int(self._pay * 1.04) # %4 increase
        
# test it
emp1 = Employee("Juan", "Perez", 50000.99)
print(emp1._pay)
emp1.apply_raise()
print(emp1._pay)

50000.99
52001


There are a few thing wrong here. We cannot access the emp1.raise_amount because it does not exist.

What if you want to increase more than 4%. 

Make a class variable.

In [32]:
class Employee:
    raise_amount = 1.04
    def __init__(self, first, last, pay):
        self._first = first
        self._last = last
        self._pay = pay
        self._email = first + '.' + last + '@weber.edu'
        
    def fullname(self):
        return "{} {}".format(self._first, self._last)
    
    def apply_raise(self):
        self._pay = int(self._pay * self.raise_amount) # %4 increase
        
# test it
emp1 = Employee("Juan", "Perez", 50000.99)
print(emp1._pay)
emp1.apply_raise()
print(emp1._pay)

50000.99
52001


In [33]:
# Access class variables through the class itself
# or an instance of the class
emp1 = Employee("Juan", "Perez", 50000.00)
emp2 = Employee("Ana", "White", 550000.00)
print(Employee.raise_amount)
print(emp1.raise_amount)
print(emp2.raise_amount)

1.04
1.04
1.04


Now, let's look at the namespace of the object

In [34]:
emp3 = Employee("Isabel", "Smith", 92000.20)
print(emp3.__dict__)

{'_first': 'Isabel', '_last': 'Smith', '_pay': 92000.2, '_email': 'Isabel.Smith@weber.edu'}


There is no **raise_amount** attribute. If we print the class

In [35]:
print(Employee.__dict__)

{'__module__': '__main__', 'raise_amount': 1.04, '__init__': <function Employee.__init__ at 0x00000172C739E7B8>, 'fullname': <function Employee.fullname at 0x00000172C739E840>, 'apply_raise': <function Employee.apply_raise at 0x00000172C739ED08>, '__dict__': <attribute '__dict__' of 'Employee' objects>, '__weakref__': <attribute '__weakref__' of 'Employee' objects>, '__doc__': None}


Now, let's see how this affects the instances:

In [36]:
emp1 = Employee("Juan", "Perez", 50000.00)
emp2 = Employee("Ana", "White", 550000.00)
Employee.raise_amount = 1.05
print(Employee.raise_amount)
print(emp1.raise_amount)
print(emp2.raise_amount)

1.05
1.05
1.05


What if you set the amount_raise through an instance?

In [37]:
emp1.raise_amount = 1.08
print(Employee.raise_amount)
print(emp1.raise_amount)
print(emp2.raise_amount)

1.05
1.08
1.05


In [39]:
print(emp1.__dict__)
print(emp2.__dict__)

{'_first': 'Juan', '_last': 'Perez', '_pay': 50000.0, '_email': 'Juan.Perez@weber.edu', 'raise_amount': 1.08}
{'_first': 'Ana', '_last': 'White', '_pay': 550000.0, '_email': 'Ana.White@weber.edu'}


In [40]:
emp2.raise_amt = 1.09
print(emp1.__dict__)
print(emp2.__dict__)

{'_first': 'Juan', '_last': 'Perez', '_pay': 50000.0, '_email': 'Juan.Perez@weber.edu', 'raise_amount': 1.08}
{'_first': 'Ana', '_last': 'White', '_pay': 550000.0, '_email': 'Ana.White@weber.edu', 'raise_amt': 1.09}


Now, we find the variable within the local namespace. This is important concept. It depends how you define it. Also, this will allow any **subclass** to overwrite this value if needed. 

In [41]:
class Employee:
    raise_amount = 1.04
    num_of_emps = 0
    def __init__(self, first, last, pay):
        self._first = first
        self._last = last
        self._pay = pay
        self._email = first + '.' + last + '@weber.edu'
        
        Employee.num_of_emps += 1
        
    def fullname(self):
        return "{} {}".format(self._first, self._last)
    
    def apply_raise(self):
        self._pay = int(self._pay * self.raise_amount) # %4 increase
        
# test 
print(Employee.num_of_emps)
emp1 = Employee("Juan", "Perez", 50000.00)
emp2 = Employee("Ana", "White", 550000.00)
print(Employee.num_of_emps)

0
2


## Class Methods and Static Methods

Regular Methods in our classes, take the **self** as the first argument(by convention).

To turn a regular method into a **class method**, use a 
decorator **@classmethod**

In [42]:
class Employee:
    raise_amount = 1.04
    num_of_emps = 0
    def __init__(self, first, last, pay):
        self._first = first
        self._last = last
        self._pay = pay
        self._email = first + '.' + last + '@weber.edu'
        
        Employee.num_of_emps += 1
        
    def fullname(self):
        return "{} {}".format(self._first, self._last)
    
    def apply_raise(self):
        self._pay = int(self._pay * self.raise_amount) # %4 increase
    
    @classmethod
    def set_raise_amt(cls, amount):
        cls.raise_amount = amount
        
# test 
emp1 = Employee("Juan", "Perez", 50000.00)
emp2 = Employee("Ana", "White", 550000.00)
print(Employee.raise_amount)
print(emp1.raise_amount)
print(emp2.raise_amount)

1.04
1.04
1.04


As you can see, they all are the same. Now, let's updat the amount to 5%

In [44]:
Employee.set_raise_amt(1.05)
print(Employee.raise_amount)
print(emp1.raise_amount)
print(emp2.raise_amount)

1.05
1.05
1.05


You can run **class methods** from an instance, but I do not see the point

In [45]:
# Set it here
emp1.set_raise_amt(1.07)
print(Employee.raise_amount)
print(emp1.raise_amount)
print(emp2.raise_amount)

1.07
1.07
1.07


### Alternative "constructors"
Use this **classmethods** to provide multiple ways to create your object.<cr>
For example, your data might come in diffrent forms:

In [46]:
emp1 = Employee("Juan", "Perez", 50000.00)
emp2 = Employee("Ana", "White", 550000.00)
# Employee info separated by hyphen
emp_str3 = "Manuel-Garcia-45000"
emp_str4 = 'James-Smith-35000'
emp_str5 = "Manu-Ginobili-42000"
# One is to split it
first, last, pay = emp_str3.split('-')
emp3 = Employee(first, last, pay)

This is a common form, you want to create an alternative way to construct your objets. 

Create a new **classmethod** by convetion these alternative initializers begin with the form **from\_** name

In [47]:
class Employee:
    raise_amount = 1.04
    num_of_emps = 0
    def __init__(self, first, last, pay):
        self._first = first
        self._last = last
        self._pay = pay
        self._email = first + '.' + last + '@weber.edu'
        
        Employee.num_of_emps += 1
        
    def fullname(self):
        return "{} {}".format(self._first, self._last)
    
    def apply_raise(self):
        self._pay = int(self._pay * self.raise_amount) # %4 increase
    
    @classmethod
    def set_raise_amt(cls, amount):
        cls.raise_amount = amount
    
    @classmethod
    def from_string(cls, emp_str):
        first, last, pay = emp_str.split('-')
        return cls(first, last, pay) # create and return new employee

In [48]:
# test it
emp4 = Employee.from_string(emp_str4)
print(emp4._email)

James.Smith@weber.edu


Now, try it with an existing package. For example: **datetime**

## Static Methods
* Instance methods pass the **self** as the first parameter
* Class methods pass the **cls** as the first parameter
* Static methods, do not pass anything automatically. They are just regular definitions. Exept, they are relevant to the class. 

Ex: You need to pass in a date, and you want to know if this is a working day. This has a logical connection to the class functionality, but does not depend on any instance or class variable.

We are also going to use a decorator **@staticmethod**<cr>
and use one **\_** as a prefix to the name

In [54]:
class Employee:
    raise_amount = 1.04
    num_of_emps = 0
    def __init__(self, first, last, pay):
        self._first = first
        self._last = last
        self._pay = pay
        self._email = first + '.' + last + '@weber.edu'
        
        Employee.num_of_emps += 1
        
    def fullname(self):
        return "{} {}".format(self._first, self._last)
    
    def apply_raise(self):
        self._pay = int(self._pay * self.raise_amount) # %4 increase
    
    @classmethod
    def set_raise_amt(cls, amount):
        cls.raise_amount = amount
    
    @classmethod
    def from_string(cls, emp_str):
        first, last, pay = emp_str.split('-')
        return cls(first, last, pay) # create and return new employee
    
    # Monday = 0, Sunday = 6
    @staticmethod
    def _is_workday(day):
        if day.weekday() == 5 or day.weekday() == 6:
            return False
        return True

In [55]:
import datetime
datetime.__file__

emp1 = Employee("Juan", "Perez", 50000.00)
emp2 = Employee("Ana", "White", 550000.00)
my_date = datetime.date(2017, 12, 19)
print(Employee._is_workday(my_date))

True


### Choosing Static or Class Method

@staticmethod  | @classmethod
---------------|------------
No access needed to either **class** or **instance** object | Requires access to the class object to call the other class methods or the constructor
Most likely an implementation detail of the class | None
May be able to moved to become a module scope function | None

Get the following file: https://icarus.cs.weber.edu/~hvalle/hafb/iso6346.py

### Static Method with Inheritance
Let's look at static method and how they behave with **inheritance**.<cr>
Unlike other languages, **static methods** can be overwritten in **subclasses**. 

Any reference in the parent class to the ParentClass name, needs to be updated to use the instance **self** instead of the class name. 

Note: see shipping.py and iso6346.py

## Class Methods with Inheratance

If you have experience in other languages, you should recognize this ability to have class methods behave **polymorphically** as a distinguished feature of Python. 

These features work beacuse the base class **dunder-init** initializer method is inherited into the subclass. 

To call base class methods, use the **super()** fuction.

## Using *args and **kwargs to accomodate extra arguments in the base class

### Creating a Read only property
Use the **@property** decorator

See: shipping.py

In [56]:
# make your attribute Read+Write
class Example:
    @property
    def p(self):
        return self._p
    
    @p.setter 
    def p(self, value):
        self._p = value